In [17]:
import pandas as pd
import numpy as np
import os

In [18]:
path = '../data/'

In [19]:
files = os.listdir(path)

In [20]:
# create dictionary of columns to keep for each data file
climate_keep = ['FD_sum_mean', 'TD_sum_mean', 'osm_id', 'zip_code', 'zip_group']

column_dict = {}
column_dict['climate_reduced_railbelt_1981_2010.csv'] = climate_keep
column_dict['climate_reduced_railbelt_2000.csv'] = climate_keep
column_dict['climate_reduced_railbelt_2010.csv'] = climate_keep
column_dict['climate_reduced_railbelt_90.csv'] = climate_keep
column_dict['climate_reduced_railbelt_recent_1991_2020.csv'] = climate_keep
column_dict['climate_reduced_railbelt_80.csv'] = climate_keep
column_dict['building_height_and_sqft_railbelt.csv'] = ['areasq_ft', 'height', 'osm_id', 'zip_code', 'zip_group']


In [21]:
def drop(csv_name, path, keep_cols):
    dataframe = pd.read_csv(path + csv_name)
    no_dups = dataframe.drop_duplicates('osm_id')
    cols_to_keep  = no_dups[keep_cols]
    
    return cols_to_keep

In [22]:
subset_dataframes = []
for file in files:
    if file in column_dict.keys():
        subset_df = drop(file, path, column_dict[file])
        subset_dataframes.append(subset_df)


In [24]:
# check unique osm_ids
for df in subset_dataframes:
    assert len(df['osm_id'].unique()) ==  len(df['osm_id'])